In [18]:
from langchain_community.document_loaders import TextLoader # Doesn't work with pandas DF, but with text file
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma # Vector database

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
import pandas as pd

books = pd.read_csv("books_cleaned.csv")

In [4]:
books.head()

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,age,title_and_subtitle,tagged_description
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,21.0,Gilead,9780002005883 A NOVEL THAT READERS and critics...
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,25.0,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,32.0,Rage of angels,"9780006178736 A memorable, mesmerizing heroine..."
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,23.0,The Four Loves,9780006280897 Lewis' work on the nature of lov...
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,37569.0,23.0,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le..."


In [14]:
books["tagged_description"].to_csv("tagged_description.txt",
                                   index = False,
                                   header = False)

In [20]:
raw_documents = TextLoader("tagged_description.txt", encoding='utf-8').load()
text_splitter = CharacterTextSplitter(chunk_size=0,chunk_overlap=0, separator="\n") 

'''
chunk_overlap: we dont want any chunk to have overlap, because they are separate title
chunk size: it first tries to look for the closest separator to the index number indicated by the chunks, 
so if it's >1 there's a chance it may not split on a new line but split on chunk size. So by setting it to 0, we make sure it's prioritize splitting on
the separator than trying to split on the chunk size
'''

documents = text_splitter.split_documents(raw_documents)

Created a chunk of size 1170, which is longer than the specified 0
Created a chunk of size 1216, which is longer than the specified 0
Created a chunk of size 375, which is longer than the specified 0
Created a chunk of size 311, which is longer than the specified 0
Created a chunk of size 483, which is longer than the specified 0
Created a chunk of size 484, which is longer than the specified 0
Created a chunk of size 962, which is longer than the specified 0
Created a chunk of size 188, which is longer than the specified 0
Created a chunk of size 845, which is longer than the specified 0
Created a chunk of size 296, which is longer than the specified 0
Created a chunk of size 197, which is longer than the specified 0
Created a chunk of size 881, which is longer than the specified 0
Created a chunk of size 1090, which is longer than the specified 0
Created a chunk of size 1191, which is longer than the specified 0
Created a chunk of size 306, which is longer than the specified 0
Create

In [21]:
documents[0]

Document(metadata={'source': 'tagged_description.txt'}, page_content='"9780002005883 A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, G

In [22]:
len(documents)

5197

In [25]:
# Create a document embedding and store them in a database

db_books = Chroma.from_documents(
    documents, 
    embedding=OpenAIEmbeddings())

In [27]:
query = "A book to teach children about nature"
docs = db_books.similarity_search(query, k=10)
docs

[Document(id='b0a06821-63c8-4d3f-ab55-998065f29ebd', metadata={'source': 'tagged_description.txt'}, page_content='"9780786808069 Children will discover the exciting world of their own backyard in this introduction to familiar animals from cats and dogs to bugs and frogs. The combination of photographs, illustrations, and fun facts make this an accessible and delightful learning experience."'),
 Document(id='aab00db2-de07-49de-bc3a-b4d4c7f2e87b', metadata={'source': 'tagged_description.txt'}, page_content='"9780786808380 Introduce your babies to birds, cats, dogs, and babies through fine art, illustration, and photographs. These books are a rare opportunity to expose little ones to a range of images on a single subject, from simple child\'s drawings and abstract art to playful photos. A brief text accompanies each image, introducing the baby to some basic -- and sometimes playful -- information about the subjects."'),
 Document(id='60c6da18-f22b-466e-b723-d1fb4f0abe73', metadata={'sourc

In [28]:
docs[0]

Document(id='b0a06821-63c8-4d3f-ab55-998065f29ebd', metadata={'source': 'tagged_description.txt'}, page_content='"9780786808069 Children will discover the exciting world of their own backyard in this introduction to familiar animals from cats and dogs to bugs and frogs. The combination of photographs, illustrations, and fun facts make this an accessible and delightful learning experience."')

In [32]:
books[books["isbn13"]== int(docs[0].page_content.split()[0].strip().strip('"'))] #split()[0] is splitting page_content based on space and return the first element (isbn13) & strip it so there's no trailing white space


,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,age,title_and_subtitle,tagged_description
3747,9780786808069,0786808063,Baby Einstein: Neighborhood Animals,Marilyn Singer;Julie Aigner-Clark,Juvenile Fiction,http://books.google.com/books/content?id=X9a4P...,Children will discover the exciting world of t...,2001.0,3.89,16.0,180.0,24.0,Baby Einstein: Neighborhood Animals,9780786808069 Children will discover the excit...


In [38]:
def retrieve_semantic_recommendations(
    query: str,
    top_k: int = 10
) -> pd.DataFrame:
    recs = db_books.similarity_search(query, k=50)

    books_list = []

    for i in range(0, len(recs)):
        books_list += [int(recs[i].page_content.split()[0].strip().strip('"'))]

    return books[books['isbn13'].isin(books_list)].head(top_k)

In [39]:
retrieve_semantic_recommendations("A book to teach children about nature")

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,age,title_and_subtitle,tagged_description
31,9780007105045,0007105045,Tree and Leaf,John Ronald Reuel Tolkien,Literary Collections,http://books.google.com/books/content?id=aPb_A...,"""The two works 'On fairy-stories' and 'Leaf by...",2001.0,4.09,176.0,2245.0,24.0,Tree and Leaf: The Homecoming of Beorhtnoth : ...,"9780007105045 ""The two works 'On fairy-stories..."
59,9780007151240,0007151241,The Family Way,Tony Parsons,Parenthood,http://books.google.com/books/content?id=dJEIx...,It should be the most natural thing in the wor...,2005.0,3.51,400.0,2095.0,20.0,The Family Way,9780007151240 It should be the most natural th...
143,9780060546571,0060546573,Three Rotten Eggs,Gregory Maguire,Juvenile Fiction,http://books.google.com/books/content?id=t2pWl...,The students of Miss Earth's class in rural Ve...,2005.0,3.74,240.0,76.0,20.0,Three Rotten Eggs,9780060546571 The students of Miss Earth's cla...
324,9780060959036,0060959037,Prodigal Summer,Barbara Kingsolver,Fiction,http://books.google.com/books/content?id=06IwG...,Barbara Kingsolver's fifth novel is a hymn to ...,2001.0,4.00,444.0,85440.0,24.0,Prodigal Summer: A Novel,9780060959036 Barbara Kingsolver's fifth novel...
400,9780062700254,0062700251,Bulfinch's Mythology,Richard P. Martin,Reference,http://books.google.com/books/content?id=eev4u...,A beautiful gift edition of Thomas Bulfinch's ...,1991.0,4.10,768.0,64.0,34.0,"Bulfinch's Mythology: The Age of the Fable, Th...",9780062700254 A beautiful gift edition of Thom...
429,9780064434980,0064434982,The Deer in the Wood,Laura Ingalls Wilder,Juvenile Fiction,http://books.google.com/books/content?id=V7YDW...,Even the youngest child can enjoy a special ad...,1999.0,4.17,32.0,302.0,26.0,The Deer in the Wood,9780064434980 Even the youngest child can enjo...
442,9780067575208,006757520X,The Sense of Wonder,Rachel Carson,Nature,http://books.google.com/books/content?id=Zee5S...,"First published more than three decades ago, t...",1998.0,4.39,112.0,1160.0,27.0,The Sense of Wonder,9780067575208 First published more than three ...
692,9780140448009,0140448004,Three Tales,Gustave Flaubert;Roger Whitehouse;Geoffrey Wall,Fiction,http://books.google.com/books/content?id=XFzga...,Features short fiction by the French naturalis...,2005.0,3.71,110.0,3050.0,20.0,Three Tales,9780140448009 Features short fiction by the Fr...
707,9780140568196,0140568190,The Giraffe and the Pelly and Me,Roald Dahl;Quentin Blake,Candy,http://books.google.com/books/content?id=J7FdI...,"A Dahl story in which the giraffe, the pelican...",2001.0,3.81,32.0,16265.0,24.0,The Giraffe and the Pelly and Me,9780140568196 A Dahl story in which the giraff...
711,9780140621624,0140621628,The Railway Children,E. Nesbit,Fiction,http://books.google.com/books/content?id=fFesd...,"When their father is sent away to prison, thre...",1995.0,4.00,212.0,178.0,30.0,The Railway Children,9780140621624 When their father is sent away t...


In [40]:
retrieve_semantic_recommendations("A book about technology")

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,age,title_and_subtitle,tagged_description
643,9780140345827,0140345825,Strange Attractors,William Sleator,Juvenile Fiction,http://books.google.com/books/content?id=DpgIA...,Max finds himself in possession of a time trav...,1991.0,3.71,176.0,226.0,34.0,Strange Attractors,9780140345827 Max finds himself in possession ...
816,9780142402931,0142402931,The Far Side of Evil,Sylvia Engdahl,Juvenile Fiction,http://books.google.com/books/content?id=7nijj...,A young girl from an advanced civilization is ...,2005.0,3.98,324.0,57.0,20.0,The Far Side of Evil,9780142402931 A young girl from an advanced ci...
917,9780156032971,015603297X,Foucault's Pendulum,Umberto Eco,Fiction,http://books.google.com/books/content?id=Ubkvm...,"Three editors, inspired by an extraordinary fa...",2007.0,3.90,623.0,44658.0,18.0,Foucault's Pendulum,"9780156032971 Three editors, inspired by an ex..."
1059,9780226817415,0226817415,From Counterculture to Cyberculture,Fred Turner,Social Science,http://books.google.com/books/content?id=wz5Em...,"In the early 1960s, computers haunted the Amer...",2006.0,3.96,327.0,323.0,19.0,From Counterculture to Cyberculture: Stewart B...,"9780226817415 In the early 1960s, computers ha..."
1141,9780307265432,0307265439,The Road,Cormac McCarthy,Fiction,http://books.google.com/books/content?id=eMq_l...,"In a novel set in an indefinite, futuristic, p...",2006.0,3.96,241.0,574745.0,19.0,The Road,"9780307265432 In a novel set in an indefinite,..."
1219,9780312313678,0312313675,The New York Times Guide to Essential Knowledge,The New York Times,Reference,http://books.google.com/books/content?id=zqkdN...,"Presents information on such subjects as art, ...",2004.0,4.09,1104.0,150.0,21.0,The New York Times Guide to Essential Knowledg...,9780312313678 Presents information on such sub...
1255,9780312423131,0312423136,Galatea 2.2,Richard Powers,Fiction,http://books.google.com/books/content?id=ttcEu...,"The protagonist, a Humanist-in-Residence at th...",2004.0,3.72,329.0,2063.0,21.0,Galatea 2.2: A Novel,"9780312423131 The protagonist, a Humanist-in-R..."
1271,9780312852535,0312852533,The Humanoids,Jack Williamson,Fiction,http://books.google.com/books/content?id=vPSl0...,A classic science fiction novel features human...,1996.0,3.75,299.0,880.0,29.0,The Humanoids: A Novel,9780312852535 A classic science fiction novel ...
1288,9780312890216,0312890214,The Starry Rift,James Tiptree,Fiction,NaN,This novel set in the far-future and filled wi...,1994.0,3.82,250.0,220.0,31.0,The Starry Rift,9780312890216 This novel set in the far-future...
1351,9780316281331,0316281336,"The World Treasury of Physics, Astronomy, and ...",Timothy Ferris;Clifton Fadiman,Education,http://books.google.com/books/content?id=zryXG...,The writings of more than 60 authors including...,1993.0,4.12,859.0,361.0,32.0,"The World Treasury of Physics, Astronomy, and ...",9780316281331 The writings of more than 60 aut...
